# MCP와 OpenAI 통합

### 데이터 흐름 설명

1. **사용자 질문**: 사용자가 시스템에 질문을 보낸다 (예: "우리 회사의 휴가 정책은 무엇인가?").
2. **OpenAI API**: OpenAI는 MCP 서버로부터 받은 사용 가능한 도구와 함께 질문을 받는다.
3. **도구 선택**: OpenAI는 질문에 기반하여 어떤 도구를 사용할지 결정한다.
4. **MCP 클라이언트**: 클라이언트는 OpenAI의 도구 호출 요청을 받아 MCP 서버로 전달한다.
5. **MCP 서버**: 서버는 요청된 도구를 실행한다 (예: 지식 베이스 데이터 검색).
6. **응답 흐름**: 도구 결과는 MCP 클라이언트를 통해 OpenAI로 다시 전달된다.
7. **최종 응답**: OpenAI는 도구 데이터를 통합하여 최종 응답을 생성한다.

## 1. 지식 베이스 (Knowledge Base) 설정

In [ ]:
import os
import json

# 데이터 디렉토리 생성
if not os.path.exists('data'):
    os.makedirs('data')

# 지식 베이스 데이터 정의
kb_data = [
    {
        "question": "What is the company's vacation policy?",
        "answer": "Employees are entitled to 20 paid vacation days per year. Unused days can be carried over up to a maximum of 5 days to the next year."
    },
    {
        "question": "What are the working hours?",
        "answer": "Standard working hours are from 9:00 AM to 6:00 PM, Monday to Friday, with a one-hour lunch break."
    },
    {
        "question": "How does the remote work policy work?",
        "answer": "Employees can work remotely up to 2 days per week with approval from their direct manager."
    }
]

# kb.json 파일 작성
with open('data/kb.json', 'w') as f:
    json.dump(kb_data, f, indent=2)

print("'data/kb.json' 파일이 성공적으로 생성되었다.")

## 2. 서버 (`server.py`)

In [ ]:
%%writefile server.py
import os
import json
from mcp.server.fastmcp import FastMCP

# MCP 서버 생성
mcp = FastMCP(
    name="Knowledge Base",
    host="0.0.0.0",  # SSE 전송 방식에서만 사용됨 (localhost)
    port=8050,  # SSE 전송 방식에서만 사용됨 (원하는 포트로 설정)
)


@mcp.tool()
def get_knowledge_base() -> str:
    """전체 지식 베이스를 형식화된 문자열로 검색한다.

    Returns:
        지식 베이스의 모든 Q&A 쌍을 포함하는 형식화된 문자열이다.
    """
    try:
        kb_path = os.path.join(os.path.dirname(__file__), "data", "kb.json")
        with open(kb_path, "r") as f:
            kb_data = json.load(f)

        # 지식 베이스를 문자열로 형식화
        kb_text = "검색된 지식 베이스는 다음과 같다:\n\n"

        if isinstance(kb_data, list):
            for i, item in enumerate(kb_data, 1):
                if isinstance(item, dict):
                    question = item.get("question", "알 수 없는 질문")
                    answer = item.get("answer", "알 수 없는 답변")
                else:
                    question = f"항목 {i}"
                    answer = str(item)

                kb_text += f"Q{i}: {question}\n"
                kb_text += f"A{i}: {answer}\n\n"
        else:
            kb_text += f"지식 베이스 내용: {json.dumps(kb_data, indent=2)}\n\n"

        return kb_text
    except FileNotFoundError:
        return "오류: 지식 베이스 파일을 찾을 수 없다"
    except json.JSONDecodeError:
        return "오류: 지식 베이스 파일에 잘못된 JSON이 있다"
    except Exception as e:
        return f"오류: {str(e)}"


# 서버 실행
if __name__ == "__main__":
    mcp.run(transport="stdio")

## 3. 클라이언트 (`client.py`)

클라이언트는 다음을 수행한다:
1. MCP 서버에 연결한다.
2. MCP 도구를 OpenAI의 함수 형식으로 변환한다.
3. OpenAI와 MCP 서버 간의 통신을 처리한다.
4. 도구 결과를 처리하고 최종 응답을 생성한다.

In [ ]:
import asyncio
import json
from contextlib import AsyncExitStack
from typing import Any, Dict, List, Optional

import nest_asyncio
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from openai import AsyncOpenAI

# 중첩 이벤트 루프를 허용하도록 nest_asyncio 적용 (Jupyter/IPython에 필요)
nest_asyncio.apply()

# .env 파일에서 환경 변수 로드 (OPENAI_API_KEY 설정 필요)
load_dotenv("../.env")

class MCPOpenAIClient:
    """MCP 도구를 사용하여 OpenAI 모델과 상호 작용하는 클라이언트다."""

    def __init__(self, model: str = "gpt-4o"):
        """OpenAI MCP 클라이언트를 초기화한다.

        Args:
            model: 사용할 OpenAI 모델이다.
        """
        # 세션 및 클라이언트 객체 초기화
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()
        self.openai_client = AsyncOpenAI()
        self.model = model
        self.stdio: Optional[Any] = None
        self.write: Optional[Any] = None

    async def connect_to_server(self, server_script_path: str = "server.py"):
        """MCP 서버에 연결한다.

        Args:
            server_script_path: 서버 스크립트의 경로다.
        """
        # 서버 설정
        server_params = StdioServerParameters(
            command="python",
            args=[server_script_path],
        )

        # 서버에 연결
        stdio_transport = await self.exit_stack.enter_async_context(
            stdio_client(server_params)
        )
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(self.stdio, self.write)
        )

        # 연결 초기화
        await self.session.initialize()

        # 사용 가능한 도구 목록 출력
        tools_result = await self.session.list_tools()
        print("\n서버에 연결되었으며 사용 가능한 도구는 다음과 같다:")
        for tool in tools_result.tools:
            print(f"  - {tool.name}: {tool.description}")

    async def get_mcp_tools(self) -> List[Dict[str, Any]]:
        """MCP 서버에서 사용 가능한 도구를 OpenAI 형식으로 가져온다.

        Returns:
            OpenAI 형식의 도구 목록이다.
        """
        tools_result = await self.session.list_tools()
        return [
            {
                "type": "function",
                "function": {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": tool.inputSchema,
                },
            }
            for tool in tools_result.tools
        ]

    async def process_query(self, query: str) -> str:
        """OpenAI와 사용 가능한 MCP 도구를 사용하여 질문을 처리한다.

        Args:
            query: 사용자 질문이다.

        Returns:
            OpenAI의 응답이다.
        """
        # 사용 가능한 도구 가져오기
        tools = await self.get_mcp_tools()

        # 초기 OpenAI API 호출
        response = await self.openai_client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": query}],
            tools=tools,
            tool_choice="auto",
        )

        # 어시스턴트의 응답 가져오기
        assistant_message = response.choices[0].message

        # 사용자 질문과 어시스턴트 응답으로 대화 초기화
        messages = [
            {"role": "user", "content": query},
            assistant_message,
        ]

        # 도구 호출이 있는 경우 처리
        if assistant_message.tool_calls:
            # 각 도구 호출 처리
            for tool_call in assistant_message.tool_calls:
                # 도구 호출 실행
                result = await self.session.call_tool(
                    tool_call.function.name,
                    arguments=json.loads(tool_call.function.arguments),
                )

                # 대화에 도구 응답 추가
                messages.append(
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": result.content[0].text,
                    }
                )

            # 도구 결과를 포함하여 OpenAI에서 최종 응답 가져오기
            final_response = await self.openai_client.chat.completions.create(
                model=self.model,
                messages=messages,
                tools=tools,
                tool_choice="none",  # 더 이상의 도구 호출 비허용
            )

            return final_response.choices[0].message.content

        # 도구 호출이 없으면 직접 응답 반환
        return assistant_message.content

    async def cleanup(self):
        """자원을 정리한다."""
        await self.exit_stack.aclose()

## 예제 실행하기

In [ ]:
async def run_example():
    """예제를 실행하는 메인 진입점이다."""
    client = MCPOpenAIClient()
    await client.connect_to_server("server.py")

    # 예제: 회사 휴가 정책에 대해 질문
    query = "What is our company's vacation policy?"
    print(f"\n질문: {query}")

    response = await client.process_query(query)
    print(f"\n응답: {response}")
    
    # 자원 정리
    await client.cleanup()

# 비동기 함수 실행
await run_example()